# Web Scraping Demonstration

This Jupyter notebook demonstrates some key techniques for scraping information from the web.

## Some Basic Rules for Web Scraping

1. Obey copyright laws.
2. Don't break other laws.
3. Avoid overkill. If you just need some information from one or two pages, the best scraping technique is just using your web browser's `copy` and `paste` features.
4. Scan your target site(s) for policies on automated web scraping and play by those rules. Look for and read the `robots.txt` file.
5. Scrape respectfully. Automated scraping can generate hundreds of requests in seconds. That can overwhelm servers and lead to crashes. You can also trigger security measures meant to block nefarious botnets. Build pauses into your scraper to avoid being a jerk.
6. Monitor your scraper. You can accidentally send a scraper or crawler down a rabbit hole, causing it to run forever and fill up your storage media. Don't do that.


## Project

Imagine that your task is to assemble a table of basic information about the universities in the Southeast Conference (SEC). You *could* go to the websites of all sixteen organizations and hunt down everything you need to know. That would take hours or even days, but you need the information **now**.

You discover that Wikipedia has [a page on the SEC](https://en.wikipedia.org/wiki/Southeastern_Conference). It has a nice table with links leading to Wikipedia pages about the individual SEC members. Hot dog! You *could* just copy and paste that table and then look for more in-depth information to copy from all sixteen individual member institution pages. That, too, would take hours, and you have better things to do with your time than clicking on links and copying and pasting information.

A quick glance at some of the individual member pages reveals that each one has a sidebar with "at a glance" information. Some of that information is exactly what you need.

Let's build a scraper that will do the following:

1. Request the main SEC page at <https://en.wikipedia.org/wiki/Southeastern_Conference>
2. Find the table on all 16 member institutions
3. Build a list of names of institutions and links to their individual pages
4. Request each of the individual pages
5. Find the "infobox" table on each individual page
6. Parse the information in the "infobox" table
7. Scrape the "infobox" information into Pandas dataframes
8. Consolidate the "infobox" information for all 16 institutions
9. Save the information in an Excel workbook

I've selected Wikipedia as the target for three reasons:

1. The information is openly available.
2. The information has a predictable structure.
3. The site handles thousands of requests every minute, so it won't be affected by some light web scraping.

## Tools

Before we do anything with a scraper, we need to have a goal in mind. The best place to start is in your web browser.

You'll need to look at the target page's raw HTML code to get an idea of what to tell your scraper to scrape. The best way to do this is to use the tools that most major web browsers make available to you.

- [Firefox](https://firefox-source-docs.mozilla.org/devtools-user/)
- [Safari](https://developer.apple.com/safari/tools/)
- [Google Chrome](https://www.google.com/chrome/dev/)
- [Microsoft Edge](https://learn.microsoft.com/en-us/microsoft-edge/devtools-guide-chromium/landing/?form=MA13LH)
- [Vivaldi](https://help.vivaldi.com/desktop/tools/developer-tools/)
- [Opera](https://www.opera.com/opera/developer)

All of these tools work in the same way. They let you move your cursor over parts of a web page and inspect the code elements responsible for structuring and rendering them.

### Python Modules

If you're working on Google Colab, the modules you need for this tutorial are already installed. In case you're not on Colab, or if you're just interested, read on …

To scrape information from the internet, you need to be able to connect to the internet. Aside from an active and reliable internet connection, you will also need the module that enables Python to request information from the internet. That module is aptly named [`requests`](https://docs.python-requests.org/en/latest/index.html). Install `requests` with `conda install anaconda::requests` or `pip install requests`.

You also need to be able to read and parse the requested information. For that, we use the more imaginatively named [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/). Install `beautifulsoup` with `conda install anaconda::beautifulsoup4` or `pip install beautifulsoup4`.

You can do a lot with those two modules, but I also recommend incorporating the standard `time` module to control how often your scraper connects to the target(s). The `regular expressions` (`re`) module is also helpful for finding and selecting the information you want to scrape. Finally, I recommend using the [`pandas`](https://pandas.pydata.org/) module to format and save your scraped information in a reusable format. Install `pandas` with `conda install anaconda::pandas` or `pip install pandas`.

In the next cell, we'll use the `import` command to, well, import these modules so that they'll be available to Python.

## Identifying Our Target

Go to <https://en.wikipedia.org/wiki/Southeastern_Conference> and use your browser's developer tools to discover the HTML code responsible for the table with the names and URL's of the SEC member institutions. It should look like this:

![Main Wikipedia Table](images/screenshot1-main-page-table.png "Main Wikipedia SEC Table")

Our data is in a `table` element. Great! How do we get it?

That's what we're going to do now.

In the next cell, we'll use the `import` command to, well, import these modules so that they'll be available to Python.


In [3]:
# Import the requests module
import requests
# We don't need everything from bs4, just BeautifulSoup
from bs4 import BeautifulSoup
# Import pandas and assign the standard shorthand name `pd` to it
import pandas as pd
# Import the standard module for regular expressions (`re`)
import re
# Import the standard module `time`
import time

## Requesting a Page

Requesting a page is quite simple: just give the URL of the page to `requests`:

In [4]:
# URL of the SEC Wikipedia page
url = 'https://en.wikipedia.org/wiki/Southeastern_Conference'

# Use `requests` to 'get' the URL
response = requests.get(url)

## Now what?

So far, we've just verified that there is a web page at a specific address. To work with the content on that page, we need to …

## Make Some Soup!

This is where `BeautifulSoup` comes in. The HTML content of the page is going to become an object (the "soup") that we'll parse with `BeautifulSoup`:

In [5]:
# Check if request was successful
if response.status_code == 200:
    # Make the "soup"
    soup = BeautifulSoup(response.content, 'html.parser')

    # Print the first 10 lines of the prettified soup
    result = soup.prettify().splitlines()
    print('\n'.join(result[0:10]))

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Southeastern Conference - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width

The "soup" is all the HTML code for the page. That's cool, but we could have just copied that from our browser. We want to get specific information from our soup, without all those HTML tags.

When we inspected the Wikipedia page, we found that our target information lives in a `table` element. But there are 39 `table` elements on this page. We don't want to work with all of them. Ideally, the tables would have unique ID tags, but they don't, so we need to look at nearby elements for something unique we can use to narrow down our search.

Is there anything in the neighborhood of our table that has a unique ID? In fact, there is! Our table is in a section with the heading "Members". Using the developer tools again, we can see that the "Members" heading is an `h3` element with `id="Members"`:

![h3](images/screenshot2-main-page-h3.png "H3 Element")

We can use that to get our target table. To do that, we'll use the `find()` method, which is one of the most common methods you'll use in scraping information from the web. Another common one is `find_all()`.

We'll use `soup.find('h3', id='Members')` to isolate that part of the soup. Then we'll use the `find_next()` method to find the `table` that's next to it:

In [6]:
# Find the heading with id="Members"
members_heading = soup.find('h3', id='Members')

# Find the next table element after this heading
target_table = members_heading.find_next('table')

# Print the first 25 lines of the table
pretty_table = target_table.prettify().splitlines()
print('\n'.join(pretty_table[0:25]))

<table class="wikitable sortable plainrowheaders" style="text-align: center;">
 <tbody>
  <tr>
   <th>
    Institution
   </th>
   <th>
    Location
   </th>
   <th>
    Founded
   </th>
   <th>
    Joined
   </th>
   <th>
    Enrollment
    <br/>
    <small>
     (fall 2023)
    </small>
    <sup class="reference" id="cite_ref-8">
     <a href="#cite_note-8">
      <span class="cite-bracket">
       [


Cool! Instead of working with all 9,789 lines of the original HTML, we're just working with the `table` that we want. Now we need to extract the information from the HTML tags.

We'll use `find_all()` on our `target_table` to find all the table's rows (`tr`). Then we'll use a `for` loop to go through all the rows and extract the data from their cells (`td`).

In [7]:
# Now you can extract data from this specific table
rows = target_table.find_all('tr')

# Create a list to store your data
table_data = []

# Loop through rows and extract data
for row in rows:
    # Extract header cells (th) and data cells (td)
    cells = row.find_all(['th', 'td'])
    row_data = [cell.get_text(strip=True) for cell in cells]
    if row_data:  # Skip empty rows
        table_data.append(row_data)
        # Print the data to the screen
        print(row_data)

['Institution', 'Location', 'Founded', 'Joined', 'Enrollment(fall 2023)[8]', 'Endowment(billions –FY24)[9]', 'Nickname', 'Colors']
['University of Alabama', 'Tuscaloosa, Alabama', '1831', '1932', '39,622', '$2.379(system-wide)', 'Crimson Tide', '']
['University of Arkansas', 'Fayetteville, Arkansas', '1871', '1992', '32,140', '$1.666', 'Razorbacks', '']
['Auburn University', 'Auburn, Alabama', '1856', '1932', '33,015', '$1.187', 'Tigers', '']
['University of Florida', 'Gainesville, Florida', '1853', '1932', '54,814', '$2.454', 'Gators', '']
['University of Georgia', 'Athens, Georgia', '1785', '1932', '41,615', '$2.056', 'Bulldogs', '']
['University of Kentucky', 'Lexington, Kentucky', '1865', '1932', '32,703[a]', '$1.979', 'Wildcats', '']
['Louisiana State University', 'Baton Rouge, Louisiana', '1860', '1932', '39,418', '$1.138(system-wide)', 'Tigers', '']
['University of Mississippi', 'University, Mississippi[b]', '1848', '1932', '24,043[c]', '$0.925', 'Rebels', '']
['Mississippi Stat

That's kind of useful, but not fantastic. We could save this information to a CSV and call it a day, but let's go further.

Those university names are in `a` elements that link to individual pages on Wikipedia. For example:

```html
<th scope="row">
    <b>
        <a href="/wiki/University_of_Alabama" title="University of Alabama">University of Alabama
        </a>
    </b>
</th>
```

You just have to add "https://en.wikipedia.org" in front of the value of `href` to have the full link. We can do that with some simple string manipulation:

In [8]:
for row in rows:
    # Use the `find_all` method to find all table header (th) cells in the row
    cells = row.find_all('th')
    univ_cell = cells[0]  # First column contains university name
    # Find the first link in the cell
    link = univ_cell.find('a')
    if link:
        # Use the `get_text` method to extract the text between the tags
        univ_name = link.get_text(strip=True)
        # Use the `get` method to extract the value of the `href` attribute
        relative_url = link.get('href')
        # Combine the base URL with the relative URL
        univ_url = "https://en.wikipedia.org" + relative_url
        # Print the university name and URL
        print(univ_name, univ_url)

University of Alabama https://en.wikipedia.org/wiki/University_of_Alabama
University of Arkansas https://en.wikipedia.org/wiki/University_of_Arkansas
Auburn University https://en.wikipedia.org/wiki/Auburn_University
University of Florida https://en.wikipedia.org/wiki/University_of_Florida
University of Georgia https://en.wikipedia.org/wiki/University_of_Georgia
University of Kentucky https://en.wikipedia.org/wiki/University_of_Kentucky
Louisiana State University https://en.wikipedia.org/wiki/Louisiana_State_University
University of Mississippi https://en.wikipedia.org/wiki/University_of_Mississippi
Mississippi State University https://en.wikipedia.org/wiki/Mississippi_State_University
University of Missouri https://en.wikipedia.org/wiki/University_of_Missouri
University of Oklahoma https://en.wikipedia.org/wiki/University_of_Oklahoma
University of South Carolina https://en.wikipedia.org/wiki/University_of_South_Carolina
University of Tennessee https://en.wikipedia.org/wiki/University_o

Now we *could* just click on those links manually and find information from those pages, but that's a lot of work. Let's just request those pages and add some more data from them.

An examination of one of those pages reveals that a bunch of information is available in a `table` with `class="infobox vcard"`:

![Infobox](images/screenshot-secondary-infobox.png "Infobox")

Let's loop through the institution URL's and scrape the "Infobox" information. Here's how we'll do it:

1. Loop through the original table and store the university names and URLs in a Python list.
2. Convert the list into a Pandas dataframe to make it easier to work with.
3. Make a Python dictionary to hold the individual dataframes that we're going to make from the universities' Wikipedia pages.
4. Request each university's page.

For each university's page, we'll do the following:

1. Make a `univ_soup` object for each page.
2. Find the `table` with `class="infobox vcard"`.
3. Extract the information from the table.
4. Create a Pandas dataframe.
5. Add the dataframe to the dictionary.

In [9]:
# Initialize a Python list to store university data
universities = []

# Skip the header row and process each row in the table body
rows = target_table.find_all('tr')[1:]  # Skip the header row

for row in rows:
    cells = row.find_all('th')
    if cells:
        # Look for the university name cell with a link
        univ_cell = cells[0]  # First column contains university name
        link = univ_cell.find('a')

        if link:
            # Use the `get_text` method to extract the text between the tags
            univ_name = link.get_text(strip=True)
            # Use the `get` method to extract the value of the `href` attribute
            relative_url = link.get('href')
            # Combine the base URL with the relative URL
            univ_url = "https://en.wikipedia.org" + relative_url
            # Make a dictionary of the university name and URL and add it to the list
            universities.append({
                'University': univ_name,
                'Wikipedia URL': univ_url
            })

# Create a dataframe with the universities and their URLs
df = pd.DataFrame(universities)

# Make a Python dictionary to store individual university dataframes
university_dataframes = {}

# Function to clean up text in infobox tables
def clean_text(text):
    # Use regular expressions to remove citations like "[1]" and "[2]"
    text = re.sub(r'\[\d+\]', '', text)
    # Use regular expressions to remove line breaks and extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Request each university's page and extract its infobox table
for index, row in df.iterrows():
    univ_name = row['University']
    univ_url = row['Wikipedia URL']

    # Create a simplified name for the dataframe variable
    simple_name = re.sub(r'[^a-zA-Z0-9]', '_', univ_name).lower()

    print(f"Processing {univ_name}...")

    # Send a GET request to the university's Wikipedia page
    univ_response = requests.get(univ_url)

    # Add a short delay to play nicely with Wikipedia's servers
    time.sleep(1)

    if univ_response.status_code == 200:
        # Make the univ_soup
        univ_soup = BeautifulSoup(univ_response.content, 'html.parser')

        # Find the infobox table
        infobox = univ_soup.find('table', class_='infobox vcard')

        if infobox:
            # Initialize a list to hold the rows from the infobox
            infobox_data = []

            # Process each row in the infobox
            for tr in infobox.find_all('tr'):
                # Check if row has th (header/label) and td (data)
                th = tr.find('th')
                td = tr.find('td')

                if th and td:
                    # Clean the text with the function we made above
                    label = clean_text(th.get_text())
                    value = clean_text(td.get_text())

                    # Append the label and value as a dictionary to the infobox_data list
                    infobox_data.append({
                        'Property': label,
                        'Value': value
                    })

            # Create a dataframe for this university
            if infobox_data:
                univ_df = pd.DataFrame(infobox_data)
                university_dataframes[simple_name] = univ_df
                # Print a message to confirm the creation of the dataframe
                print(f"Created DataFrame for {univ_name} with {len(univ_df)} entries")
            else:
                # Print a message if no data was found in the infobox
                print(f"No infobox data found for {univ_name}")
        else:
            # Print a message if no infobox was found
            print(f"No infobox found for {univ_name}")
    else:
        # Print a message if the request was not successful
        print(f"Failed to retrieve {univ_name} page. Status code: {univ_response.status_code}")

# Display the first few entries of each university dataframe
for univ_name, univ_df in university_dataframes.items():
    print(f"\n{univ_name} dataframe:")
    display(univ_df.head())

Processing University of Alabama...
Created DataFrame for University of Alabama with 19 entries
Processing University of Arkansas...
Created DataFrame for University of Arkansas with 24 entries
Processing Auburn University...
Created DataFrame for Auburn University with 23 entries
Processing University of Florida...
Created DataFrame for University of Florida with 26 entries
Processing University of Georgia...
Created DataFrame for University of Georgia with 24 entries
Processing University of Kentucky...
Created DataFrame for University of Kentucky with 23 entries
Processing Louisiana State University...
Created DataFrame for Louisiana State University with 32 entries
Processing University of Mississippi...
Created DataFrame for University of Mississippi with 14 entries
Processing Mississippi State University...
Created DataFrame for Mississippi State University with 21 entries
Processing University of Missouri...
Created DataFrame for University of Missouri with 25 entries
Processing

,Property,Value
0,Type,Public research university
1,Established,"December 18, 1820; 204 years ago (1820-12-18)"
2,Parent institution,University of Alabama System
3,Accreditation,SACS
4,Academic affiliations,ORAUURASea-grantSpace-grant



university_of_arkansas dataframe:


,Property,Value
0,Former names,Arkansas Industrial University (1871–1899)
1,Motto,Veritate duce progredi (Latin)
2,Motto in English,"""To Advance with Truth as our Leader"""
3,Type,Public land-grant research university
4,Established,"March 27, 1871; 153 years ago (1871-03-27)"



auburn_university dataframe:


...

,Property,Value
0,Former names,The University of Texas(1881–1967)
1,Motto,Disciplina Praesidium Civitatis (Latin)
2,Motto in English,"""Education is the Guardian of the State""[a]"
3,Type,Public research university
4,Established,"September 15, 1883; 141 years ago (1883-09-15)"



texas_a_m_university dataframe:


,Property,Value
0,Former names,State Agricultural and Mechanical College of T...
1,Type,Public land-grant research senior military uni...
2,Established,1876; 149 years ago (1876)[note 2]
3,Parent institution,Texas A&M University System
4,Accreditation,SACS



vanderbilt_university dataframe:


,Property,Value
0,Former name,Central University(1873–1877)
1,Motto,Crescere aude (Latin)
2,Motto in English,"""Dare to grow"""
3,Type,Private research university
4,Established,1873; 152 years ago (1873)


Fantastic! But you may have noticed that each dataframe has different kinds of information. Let's see about making one dataframe with only the information in common among all the SEC institutions.

First, we need to find the keys for each of the universities so that we can examine their individual dataframes:

In [10]:
# Print all available university keys in the dictionary
print("Available universities:")
for key in university_dataframes.keys():
    print(f"- {key}")

Available universities:
- university_of_alabama
- university_of_arkansas
- auburn_university
- university_of_florida
- university_of_georgia
- university_of_kentucky
- louisiana_state_university
- university_of_mississippi
- mississippi_state_university
- university_of_missouri
- university_of_oklahoma
- university_of_south_carolina
- university_of_tennessee
- university_of_texas_at_austin
- texas_a_m_university
- vanderbilt_university


Let's look at the University of Alabama's dataframe.

In [11]:
# Access and display the University of Alabama's DataFrame
alabama_df = university_dataframes['university_of_alabama']
# Display Alabama's DataFrame
alabama_df

,Property,Value
0,Type,Public research university
1,Established,"December 18, 1820; 204 years ago (1820-12-18)"
2,Parent institution,University of Alabama System
3,Accreditation,SACS
4,Academic affiliations,ORAUURASea-grantSpace-grant
5,Endowment,$1.22 billion (2023)(UA only)$2.09 billion (20...
6,President,Stuart R. Bell
7,Academic staff,"1,986 (1,517 full-time & 469 part-time)"
8,Students,"39,622 (fall 2023)"
9,Undergraduates,"33,435 (fall 2023)"


We can use some built-in methods in Pandas to get basic information about each dataframe. For example:

In [12]:
# To check the DataFrame's shape (rows, columns)
print(f"\nDataFrame dimensions: {alabama_df.shape}")

# To get a statistical summary of the DataFrame
print("\nDataFrame info:")
print(alabama_df.info())


DataFrame dimensions: (19, 2)

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Property  19 non-null     object
 1   Value     19 non-null     object
dtypes: object(2)
memory usage: 432.0+ bytes
None


We'll now get a list of all the properties, then we'll use the `set()` and `intersection()` methods to find the list of properties common to all the universities.

In [13]:
# Get a list of all properties for each university
university_properties = {}
for univ_name, univ_df in university_dataframes.items():
    university_properties[univ_name] = set(univ_df['Property'].tolist())

# Find the intersection of all sets of properties
all_universities = list(university_properties.keys())
if all_universities:
    common_properties = university_properties[all_universities[0]].copy()
    # Use the `intersection` method to find the common properties
    for univ_name in all_universities[1:]:
        common_properties = common_properties.intersection(university_properties[univ_name])

    # Sort the list and display it
    common_properties_list = sorted(list(common_properties))

    print(f"Properties common to all {len(all_universities)} universities:")
    for prop in common_properties_list:
        print(f"- {prop}")
    print(f"\nTotal common properties: {len(common_properties_list)}")
else:
    print("No university data found.")

Properties common to all 16 universities:
- Campus
- Endowment
- Established
- Location
- Nickname
- Sporting affiliations
- Students
- Type
- Website

Total common properties: 9


Now we'll make a new dataframe with the universities as rows and their common properties as columns.

In [14]:
# Create a new dataframe with universities as rows and common properties as columns
consolidated_data = []

for univ_name, univ_df in university_dataframes.items():
    # Start with the university name
    univ_data = {'University': univ_name}

    # Add each common property value
    for prop in common_properties_list:
        property_row = univ_df[univ_df['Property'] == prop]
        if not property_row.empty:
            univ_data[prop] = property_row['Value'].iloc[0]
        else:
            univ_data[prop] = None

    consolidated_data.append(univ_data)

# Create the consolidated dataframe
consolidated_df = pd.DataFrame(consolidated_data)

# Clean up university names for display (remove underscores, capitalize words)
consolidated_df['University'] = consolidated_df['University'].apply(
    lambda x: ' '.join(word.capitalize() for word in x.replace('_', ' ').split())
)

# Display the consolidated DataFrame
print("Consolidated University DataFrame:")
display(consolidated_df)

Consolidated University DataFrame:


,University,Campus,Endowment,Established,Location,Nickname,Sporting affiliations,Students,Type,Website
0,University Of Alabama,"Small city, 1,970 acres (8.0 km2)",$1.22 billion (2023)(UA only)$2.09 billion (20...,"December 18, 1820; 204 years ago (1820-12-18)","Tuscaloosa, Alabama, United States33°12′39″N 8...",Crimson Tide,NCAA Division I FBS – SEC,"39,622 (fall 2023)",Public research university,www.ua.edu
1,University Of Arkansas,"Small city, 412 acres (1.67 km2)",$1.7 billion (FY 2021),"March 27, 1871; 153 years ago (1871-03-27)","Fayetteville, Arkansas, United States36°04′07″...",Razorbacks,NCAA Division I FBS – SEC,"32,140 (fall 2023)",Public land-grant research university,uark.edu
2,Auburn University,"Small City, 1,841 acres (7.45 km2)",$1.3 billion (2024),"February 7, 1856; 169 years ago (1856-02-07)","Auburn, Alabama, United States",Tigers,NCAA Division I FBS – SEC,"34,195",Public land-grant research university,auburn.edu
3,University Of Florida,"Midsize city, 2,000 acres (810 ha)",$2.337 billion (2023),"January 6, 1853;172 years ago (1853-01-06)[not...","Gainesville, Florida, United States29°38′51″N ...",Gators,NCAA Division I FBS – SECBig 12,"54,814 (fall 2023)",Public land-grant research university,ufl.edu
4,University Of Georgia,"Midsize city / College town, 762 acres (3.08 k...",$1.82 billion (2023),"January 27, 1785; 240 years ago (1785-01-27)","Athens, Georgia, United States33°57′21″N 83°22...",Bulldogs,NCAA Division I FBS – SEC,"40,607 (fall 2022)",Public flagship land-grant research university,uga.edu
5,University Of Kentucky,"Large City, 784 acres (3.17 km2)",$2.13 billion (2023),"February 22, 1865; 160 years ago (February 22,...","Lexington, Kentucky, United States38°01′57″N 8...",Wildcats,NCAA Division I FBS – SECC-USAGARC,"35,952 (fall 2024)",Public land-grant research university,uky.edu
6,Louisiana State University,"Midsize city, 4,925 acres (1,993 ha)",$664.20 million (2023)(LSU only)$1.06 billion ...,"January 2, 1860; 165 years ago (January 2, 1860)","Baton Rouge, Louisiana, United States30°24′52″...",Tigers and Lady Tigers,NCAA Division I FBS – SECCCSA,"37,354 (fall 2022)",Public land-grant research university,lsu.edu
7,University Of Mississippi,"Remote town, 3,497 acres (14.15 km2)",$962 million (2024),"February 24, 1844; 181 years ago (February 24,...","University, Mississippi, 38677",Rebels,NCAA Division I FBS – SECPRC,"24,710 (for 2023-2024 year)",Public research university,olemiss.edu
8,Mississippi State University,"Remote town, 4,200 acres (17 km2)",$894.5 million (2024),"February 28, 1878; 147 years ago (February 28,...","Mississippi State, Mississippi, United States3...",Bulldogs,NCAA Division I FBS – SEC,"23,150 (fall 2024)",Public land-grant research university,msstate.edu
9,University Of Missouri,"Midsize city, 1,262 acres (511 ha)Total, 19,26...",$1.42 billion (2023)(MU only)$2.24 billion (20...,"February 11, 1839; 186 years ago (1839-02-11)","Columbia, Missouri, United States38°56′43″N 92...",Tigers,NCAA Division I FBS – SECBig 12,"31,543 (fall 2024)",Public land-grant research university,missouri.edu


Finally, let's save the dataframe as an Excel workbook and a CSV file.

In [15]:
# Save the dataframe to an Excel file
consolidated_df.to_excel('sec_universities.xlsx', index=False)
# Save the dataframe to a CSV file
consolidated_df.to_csv('sec_universities.csv', index=False)

## Next Steps

Some of that data is still in need of formatting and cleaning. You could use a tool like [OpenRefine](https://openrefine.org/) to do that. And if you need help, you can come to DH Office Hours or contact one of the fantastic experts in University Libraries' [Digital Scholarship and Data Services](https://libraries.ou.edu/units/digital-scholarship-and-data-services#:~:text=Digital%20Scholarship%20%40%20OU%20Libraries%20supports%20collaborative%2C%20cross-disciplinary,that%20build%20on%20traditional%20research%20and%20teaching%20methods.). They offer one-on-one consultations and on-request workshops. They're great!

You might also run into the challenge of scraping content from dynamic websites, sites with pagination, password protection, or some other barrier. There are techniques for handling all those scenarios and more.